# Markov networks

In this notebook, we will have a taste of Markov networks (MNs). MNs have two building blocks too: an undirected  graph structure and a set of factors.

We will use the classical **misconception example**, which has the following graph structure:

<img src="images/misconception_graph.png" style="width:200px" />

It is about a group of students (Alice, Bob, Charles and Debbie) that usually study together. The edges in the graph represent the established relationships. The lack of an edge between two nodes represents the lack of relationship.

We have a factor for each edge (in this MN, maximal cliques):

<img src="images/misconception_factors.png" style="height:150px" />

Note that these factors are not probability distributions!

Let us interpret these values. For example, B and C tend to agree with each other: having the same value (factor $\phi_2$) is rewarded with a large value, whereas having different values is not. Agreement is also observed between A and B, but the strength is much smaller (we are not that sure about it). On the contrary, C and D usually disagree: note the high value assigned to combinations where the variables take different value ($\phi_3$). 


The factorization of this MN is given by the product of the four pairwise factors and a normalizing partition function ($\Theta$):

$$p(A,B,C,D)=\frac{1}{\Theta}\phi_{1}(A,B)\phi_{2}(B,C)\phi_{3}(C,D)\phi_{4}(D,A)$$

Thus, the factors can be represented again as matrices, but they do not sum up to 1:

In [1]:
import numpy as np

phi_1 = np.array([[30, 5],
                  [1, 10]])
np.sum(phi_1, axis=0)

array([31, 15])

All the basic operations are similar to the ones used with Bayesian networks. Let us see for example how we could implement a product of factors.

In [2]:
phi_2 = np.array([[100, 1],
                  [1, 100]])

prod = np.zeros(2**3)
for a in [0,1]:
    for b in [0,1]:
        for c in [0,1]:
            prod[np.ravel_multi_index([a,b,c],(2,2,2))] = phi_1[a,b]*phi_2[b,c]
prod

array([3.e+03, 3.e+01, 5.e+00, 5.e+02, 1.e+02, 1.e+00, 1.e+01, 1.e+03])

Note always that we are not dealing with distributions anymore. Thus, if we want to infer a probability distribution from a MN we need to normalize. That is the role of the partition function ($\Theta$): to transform this product of factors into a probability distribution useful for reasoning.


## Setting up our model with pgmpy

We need to codify both elements: the undirected graph structure and the factors. 

First of all, we need to import the necessary functions from `pgmpy`:

In [3]:
from pgmpy.factors.discrete import DiscreteFactor # this is the library's implementation of a factor
from pgmpy.models import MarkovModel              # and the class for Markov network models

### Set the structure

Now, we want to specify that we are constructing a Markov network with a set of undirected edges as follows:

In [6]:
H = MarkovModel()
H.add_nodes_from(['Alice', 'Bob', 'Charles', 'Debbie'])
### YOUR CODE HERE ###  (Add missing edges that can be identified in the image)
H.add_edges_from([('Alice', 'Bob'), ('Bob', 'Charles'), ('Charles', 'Debbie'), ('Debbie', 'Alice')])

print('Check if node `Alice´ is in the graph:', ('Alice' in H)) # check if node in graph
print('Number of nodes in the graph:',len(H))

Check if node `Alice´ is in the graph: True
Number of nodes in the graph: 4


C:\Users\loren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pgmpy\models\MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(


With this simple code, we specify that there are four variables, each of them representing a student, and that there is an undirected edge connecting <i>Alice</i> and <i>Bob</i>, <i>Bob</i> and <i>Charles</i>, <i>Charles</i> and <i>Debbie</i>, and finally <i>Debbie</i> and <i>Alice</i>.


### Set up the factor potentials

Once the structure has been defined, we need to codify the factors. Firstly, the factor values assigned to all the posible combinations of values for variables <i>Alice</i> and <i>Bob</i> are:


In [7]:
factorAB = DiscreteFactor(['Alice', 'Bob'], cardinality=[2, 2],
                          values=[30, 5, 1, 10])


Following the same definition the other three factors are:


In [8]:
#### YOUR CODE HERE #### (Filling in the information from misconception_factors.png)
factorBC = DiscreteFactor(['Bob', 'Charles'], cardinality=[2, 2],
                          values=[100, 1, 1, 100])

factorCD = DiscreteFactor(['Charles', 'Debbie'], cardinality=[2, 2],
                          values=[1, 100, 100, 1])

factorDA = DiscreteFactor(['Debbie', 'Alice'], cardinality=[2, 2],
                          values=[100, 1, 1, 100])


Once the potential factors are defined, we only have to include them into the model:


In [9]:
H.add_factors(factorAB,factorBC,factorCD,factorDA)


Let us examine our model:


In [10]:
print('Is the model consistent?',H.check_model())

Is the model consistent? True



Another important characteristic of Markov networks is the necessity of a partition function to build a joint probability distribution from the product of factors. This constant, represented in the above equation as $\Theta$, is defined as the sum of the product for all the possible value combinations:
$$\Theta=\sum_{a,b,c,d}\phi_{AB}(A,B)\phi_{BC}(B,C)\phi_{CD}(C,D)\phi_{DA}(D,A)$$

Its value can be obtained as follow:


In [11]:
print('Partition function value (constant):',H.get_partition_function())

Partition function value (constant): 7201840.0


## Using our Markov network

We have already built our model. It is time to use it!

We can easily find all the local <b>independencies</b> in the model:


In [12]:
print('Set of conditional independencies in the MN:\n',H.get_local_independencies())

Set of conditional independencies in the MN:
 (Alice ⟂ Charles | Debbie, Bob)
(Bob ⟂ Debbie | Alice, Charles)
(Charles ⟂ Alice | Debbie, Bob)
(Debbie ⟂ Bob | Alice, Charles)



In few words, two opposite edges in the graph are conditionally independent given the other 2. Note that the independencies given by the ```get_local_independencies()``` method are repeated.


This set is obtained, given each single random variable $X_i$, by looking for all the variables out of the <b>Markov Blanket</b> of $X_i$ as we know that those are independent from $X_i$ given its Mb:

$$X_i\perp \{V \backslash X_i\backslash MB(X_i)\} |MB(X_i)$$

Thus, we might be interested in knowing the Markov Blanket of <i>Alice</i>:

In [13]:
print('Markov Blanket of `Alice´:', list(H.markov_blanket('Alice')))

Markov Blanket of `Alice´: ['Bob', 'Debbie']


As expected, the Markov Blanket of 'Alice' in our Markov Network is composed by its neighbors.


Regarding the set of (conditional) independencies that are encoded by a Markov network and its equivalent, if possible, Bayesian network, we can transform the MN to a BN and observe the produced structure:


In [14]:
bn = H.to_bayesian_model()
print('Set of directed edges in the BN:\n',bn.edges())

Set of directed edges in the BN:
 [('Charles', 'Alice'), ('Charles', 'Debbie'), ('Charles', 'Bob'), ('Alice', 'Debbie'), ('Alice', 'Bob')]


C:\Users\loren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pgmpy\models\MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
C:\Users\loren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pgmpy\models\MarkovNetwork.py:448: RuntimeWarning: invalid value encountered in long_scalars
  S_by_C = {key: S[key] / C[key] for key in S}


In [15]:
print('Set of conditional independencies in the BN:\n',bn.get_independencies())

Set of conditional independencies in the BN:
 (Debbie ⟂ Bob | Alice, Charles)
(Bob ⟂ Debbie | Alice, Charles)



and observe that this DAG $G$ is an I-map of the undirected graph $H$, $I(G)\subseteq I(H)$, but does not represent the same set of conditional independence statements. **What is missing**? Why?

It must be observed that a new edge was added when transforming the Markov Network into a Bayesian Network: ('Charles, 'Alice'). Therefore, they are not independent given the rest anymore.

We have seen in class that in order to obtain a Markov Network from a Bayesian Network we just need to moralize the graph.

If it is desired to follow the opposite direction, we need to compute the minimal chordal graph of the given Markov Network. (refer to https://cedar.buffalo.edu/~srihari/CSE574/Chap8/8.7-FromBNtoMN.pdf)

Also, if ('Charles', 'Alice') was not added, we would have had two v-structures in our Bayesian Network, with Charles and Alice edges as parents in both of them.

<hr /> 

## Asking our Markov network

Later in this course, we will know different approaches for inference in PGMs. However, let us consider the approach known as <i>Variable Elimination</i> to observe how we can perform different queries.

In [16]:
from pgmpy.inference import VariableElimination
inference_with_H = VariableElimination(H)


We can obtain the marginal probability distributions of different variables:


In [17]:
prob_alice = inference_with_H.query(variables = ['Alice'])
print(prob_alice.normalize(False))

Eliminating: Bob: 100%|██████████| 3/3 [00:00<?, ?it/s]    

+----------+--------------+
| Alice    |   phi(Alice) |
+==========+==============+
| Alice(0) |       0.8194 |
+----------+--------------+
| Alice(1) |       0.1806 |
+----------+--------------+


Note that we need to normalize to obtain a probability distribution! What will we see if we don't?

Similarly, the marginal probability distributions of <i>Bob</i> is:


In [18]:
prob_bob = inference_with_H.query(variables = ['Bob'])
print(prob_bob.normalize(False))

Eliminating: Charles: 100%|██████████| 3/3 [00:00<00:00, 680.41it/s]

+--------+------------+
| Bob    |   phi(Bob) |
+========+============+
| Bob(0) |     0.2639 |
+--------+------------+
| Bob(1) |     0.7361 |
+--------+------------+



It is interesting to observe that, although Alice and Bob are likely to agree according to factor potential $\phi_1(A,B)$, the marginal probability distributions of both are not. This is a consequence of the rest of the relationships in the model and emphasizes the conceptual distance between potential factor and joint or conditional probability distributions.

The most common use of inference techniques is for propagating the evidence gathered from certain observations. We can calculate the marginal probability <i>Alice</i> and <i>Charles</i> given that <i>Bob</i> is unaware of the misconception:


In [19]:
prob_Alice_Bob_unaware = inference_with_H.query(variables = ['Alice','Debbie'], 
                                              evidence = {'Bob':0})
print(prob_Alice_Bob_unaware.normalize(False))

Eliminating: Charles: 100%|██████████| 1/1 [00:00<00:00, 984.58it/s]

+-----------+----------+---------------------+
| Debbie    | Alice    |   phi(Debbie,Alice) |
+===========+==========+=====================+
| Debbie(0) | Alice(0) |              0.3157 |
+-----------+----------+---------------------+
| Debbie(0) | Alice(1) |              0.0001 |
+-----------+----------+---------------------+
| Debbie(1) | Alice(0) |              0.1579 |
+-----------+----------+---------------------+
| Debbie(1) | Alice(1) |              0.5263 |
+-----------+----------+---------------------+



It is also possible to codify evidence in more than one variable. For example, we can calculate the marginal probability <i>Alice</i> and <i>Debbie</i> given that <i>Bob</i> is unaware of the misconception but <i>Charles</i> is not, which brings more certainty to the model:


In [20]:
prob_Alice_Bob_disagree_Charles = inference_with_H.query(variables = ['Alice','Debbie'], 
                                              evidence = {'Bob':0,'Charles':1})
print(prob_Alice_Bob_disagree_Charles.normalize(False))

0it [00:00, ?it/s]

+-----------+----------+---------------------+
| Debbie    | Alice    |   phi(Debbie,Alice) |
+===========+==========+=====================+
| Debbie(0) | Alice(0) |              0.9992 |
+-----------+----------+---------------------+
| Debbie(0) | Alice(1) |              0.0003 |
+-----------+----------+---------------------+
| Debbie(1) | Alice(0) |              0.0001 |
+-----------+----------+---------------------+
| Debbie(1) | Alice(1) |              0.0003 |
+-----------+----------+---------------------+



We know from the previous discussion that $D\perp B \mid A,C$. We can experience that: let us imagine that we do know the answer of <i>Alice</i> and <i>Charles</i>. The probability of each possible answer of Debbie is:


In [21]:
prob_Debbie_st_Alice_agree_Charles = inference_with_H.query(variables = ['Debbie'], 
                                                            evidence = {'Alice':1,'Charles':1})
print(prob_Debbie_st_Alice_agree_Charles.normalize(False))

Eliminating: Bob: 100%|██████████| 1/1 [00:00<00:00, 892.79it/s]

+-----------+---------------+
| Debbie    |   phi(Debbie) |
+===========+===============+
| Debbie(0) |        0.5000 |
+-----------+---------------+
| Debbie(1) |        0.5000 |
+-----------+---------------+



Does any new evidence about the answer of <i>Bob</i> contribute to update the distribution of Debbie's answer?


In [22]:
# Bob is unaware
prob_Debbie_st_Alice_agree_Charles_indBob0 = inference_with_H.query(variables = ['Debbie'], 
                                                                    evidence = {'Alice':1,'Charles':1,'Bob':0})
print(prob_Debbie_st_Alice_agree_Charles_indBob0.normalize(False))

# Bob is aware
prob_Debbie_st_Alice_agree_Charles_indBob1 = inference_with_H.query(variables = ['Debbie'], 
                                                                    evidence = {'Alice':1,'Charles':1,'Bob':1})
print(prob_Debbie_st_Alice_agree_Charles_indBob1.normalize(False))

0it [00:00, ?it/s]


+-----------+---------------+
| Debbie    |   phi(Debbie) |
+===========+===============+
| Debbie(0) |        0.5000 |
+-----------+---------------+
| Debbie(1) |        0.5000 |
+-----------+---------------+


0it [00:00, ?it/s]

+-----------+---------------+
| Debbie    |   phi(Debbie) |
+===========+===============+
| Debbie(0) |        0.5000 |
+-----------+---------------+
| Debbie(1) |        0.5000 |
+-----------+---------------+


As it could have been expected, inputting information about Bob has no impact because Debbie and Bob are conditionally independent given Alice and Charles.

<hr />

## Exercises

In order to perform these exercises, we just need to imitate the example queries we just saw and adapt them to the questions.

- Which is the probability that Charles knows about the misconception given that Alice is unaware?



In [24]:
prob_Charles_with_Alice_unaware = inference_with_H.query(variables = ['Charles'], 
                                              evidence = {'Alice':0})
print(prob_Charles_with_Alice_unaware.normalize(False))

Eliminating: Bob: 100%|██████████| 2/2 [00:00<00:00, 955.53it/s]

+------------+----------------+
| Charles    |   phi(Charles) |
+============+================+
| Charles(0) |         0.1018 |
+------------+----------------+
| Charles(1) |         0.8982 |
+------------+----------------+



- Which is the probability that Charles knows about the misconception given that Bob and Alice are unaware?



In [25]:
prob_Charles_with_Alice_Bob_unaware = inference_with_H.query(variables = ['Charles'], 
                                              evidence = {'Alice':0, 'Bob':0})
print(prob_Charles_with_Alice_Bob_unaware.normalize(False))

Eliminating: Debbie: 100%|██████████| 1/1 [00:00<00:00, 916.99it/s]

+------------+----------------+
| Charles    |   phi(Charles) |
+============+================+
| Charles(0) |         0.6666 |
+------------+----------------+
| Charles(1) |         0.3334 |
+------------+----------------+


- Which is the probability that Charles knows about the misconception given that the rest of them (Alice, Bob, and Debbie) are unaware?


In [26]:
prob_Charles_with_all_unaware = inference_with_H.query(variables = ['Charles'], 
                                              evidence = {'Alice':0, 'Bob':0, 'Debbie':0})
print(prob_Charles_with_all_unaware.normalize(False))

0it [00:00, ?it/s]

+------------+----------------+
| Charles    |   phi(Charles) |
+============+================+
| Charles(0) |         0.5000 |
+------------+----------------+
| Charles(1) |         0.5000 |
+------------+----------------+


As Charles and Alice are independent given by Bob and Debbie:

In [30]:
prob_Charles_1 = inference_with_H.query(variables = ['Charles'], 
                                              evidence = {'Alice':1, 'Bob':0, 'Debbie':0})
print(prob_Charles_1.normalize(False))

0it [00:00, ?it/s]

+------------+----------------+
| Charles    |   phi(Charles) |
+============+================+
| Charles(0) |         0.5000 |
+------------+----------------+
| Charles(1) |         0.5000 |
+------------+----------------+


However, it can be seen that the other two do really add information to the probability distribution

In [31]:
prob_Charles_2 = inference_with_H.query(variables = ['Charles'], 
                                              evidence = {'Alice':0, 'Bob':1, 'Debbie':0})
print(prob_Charles_2.normalize(False))

0it [00:00, ?it/s]

+------------+----------------+
| Charles    |   phi(Charles) |
+============+================+
| Charles(0) |         0.0001 |
+------------+----------------+
| Charles(1) |         0.9999 |
+------------+----------------+


In [32]:
prob_Charles_3 = inference_with_H.query(variables = ['Charles'], 
                                              evidence = {'Alice':0, 'Bob':0, 'Debbie':1})
print(prob_Charles_3.normalize(False))

0it [00:00, ?it/s]

+------------+----------------+
| Charles    |   phi(Charles) |
+============+================+
| Charles(0) |         0.9999 |
+------------+----------------+
| Charles(1) |         0.0001 |
+------------+----------------+


In [36]:
prob_Charles_3 = inference_with_H.query(variables = ['Charles'], 
                                              evidence = {'Alice':0, 'Bob':1, 'Debbie':1})
print(prob_Charles_3.normalize(False))

0it [00:00, ?it/s]

+------------+----------------+
| Charles    |   phi(Charles) |
+============+================+
| Charles(0) |         0.5000 |
+------------+----------------+
| Charles(1) |         0.5000 |
+------------+----------------+
